In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding
from tensorflow.keras.models import Model
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import gensim
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
import random

nltk.download('wordnet', "/kaggle/working/nltk_data/")
nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
nltk.data.path.append("/kaggle/working/nltk_data/")

2024-05-17 16:19:21.911844: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 16:19:21.911939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 16:19:22.071037: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data/...
Archive:  /kaggle/working/nltk_data/corpora/wordnet.zip
   creating: /kaggle/working/nltk_data/corpora/wordnet/
  inflating: /kaggle/working/nltk_data/corpora/wordnet/lexnames  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adv  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/nltk_data/c

In [2]:
df= pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=['target', 'id', 'date', 'flag', 'user', 'text'])
df

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [3]:
data = df[['target', 'text']]
data

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
sentences = data['text']
sentences

0          @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          is upset that he can't update his Facebook by ...
2          @Kenichan I dived many times for the ball. Man...
3            my whole body feels itchy and like its on fire 
4          @nationwideclass no, it's not behaving at all....
                                 ...                        
1599995    Just woke up. Having no school is the best fee...
1599996    TheWDB.com - Very cool to hear old Walt interv...
1599997    Are you ready for your MoJo Makeover? Ask me f...
1599998    Happy 38th Birthday to my boo of alll time!!! ...
1599999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: text, Length: 1600000, dtype: object

In [5]:
target = data['target']
target

0          0
1          0
2          0
3          0
4          0
          ..
1599995    4
1599996    4
1599997    4
1599998    4
1599999    4
Name: target, Length: 1600000, dtype: int64

In [6]:
print("length of the sentances: ",len(sentences))
print("length of the target: ",len(target))

length of the sentances:  1600000
length of the target:  1600000


In [7]:
def preprocess(review):
    review = re.sub('[^a-zA-Z]', ' ',review)
    review = review.lower()
    
    review_minus_sw = []
    stop_words = stopwords.words('english')
    review = review.split()
    review = [review_minus_sw.append(word) for word in review if word not in stop_words]
    review = ' '.join(review_minus_sw)
    
    lemmatizer = WordNetLemmatizer()
    review = review.split()
    review = [lemmatizer.lemmatize(w) for w in review]
    review = ' '.join(review)
    
    return review

In [8]:
data.loc[:, 'text'] = data['text'].apply(preprocess)
data

,target,text
0,0,switchfoot http twitpic com zl awww bummer sho...
1,0,upset update facebook texting might cry result...
2,0,kenichan dived many time ball managed save res...
3,0,whole body feel itchy like fire
4,0,nationwideclass behaving mad see
...,...,...
1599995,4,woke school best feeling ever
1599996,4,thewdb com cool hear old walt interview http b...
1599997,4,ready mojo makeover ask detail
1599998,4,happy th birthday boo alll time tupac amaru sh...


In [9]:
data['target']=target.replace(4,1)
data

/tmp/ipykernel_34/1137292383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target']=target.replace(4,1)


,target,text
0,0,switchfoot http twitpic com zl awww bummer sho...
1,0,upset update facebook texting might cry result...
2,0,kenichan dived many time ball managed save res...
3,0,whole body feel itchy like fire
4,0,nationwideclass behaving mad see
...,...,...
1599995,1,woke school best feeling ever
1599996,1,thewdb com cool hear old walt interview http b...
1599997,1,ready mojo makeover ask detail
1599998,1,happy th birthday boo alll time tupac amaru sh...


In [10]:
df_train,df_test,y_train,y_test=train_test_split(data['text'],data['target'],test_size=0.2,random_state=42, shuffle=True)
print('DF Train Shape: ',df_train.shape)
print('DF Test Shape: ',df_test.shape)
print('Y Train Shape: ',y_train.shape)
print('Y Test Shape: ',y_test.shape)

DF Train Shape:  (1280000,)
DF Test Shape:  (320000,)
Y Train Shape:  (1280000,)
Y Test Shape:  (320000,)


In [11]:
y_train

1374558    1
1389115    1
1137831    1
790714     0
1117911    1
          ..
259178     0
1414414    1
131932     0
671155     0
121958     0
Name: target, Length: 1280000, dtype: int64

In [12]:
y_test

541200     0
750        0
766711     0
285055     0
705995     0
          ..
839535     1
1023175    1
1349377    1
1086942    1
158976     0
Name: target, Length: 320000, dtype: int64

In [13]:
max_words=10000
tokenizer=Tokenizer(max_words)
tokenizer.fit_on_texts(df_train)
sequence_train=tokenizer.texts_to_sequences(df_train)
sequence_test=tokenizer.texts_to_sequences(df_test)

In [14]:
T=100
data_train=pad_sequences(sequence_train,maxlen=T)
print(data_train.shape)
data_test=pad_sequences(sequence_test,maxlen=T)
print(data_test.shape)

(1280000, 100)
(320000, 100)


In [15]:
word2vec=tokenizer.word_index
V=len(word2vec)
V

496343

In [16]:
dim=100
input_shape=Input((T,))
x=Embedding(V+1,dim)(input_shape)
x=Conv1D(32,3,activation='relu')(x)
x=MaxPooling1D(3)(x)
x=Conv1D(64,3,activation='relu')(x)
x=MaxPooling1D(3)(x)
x=Conv1D(128,3,activation='relu')(x)
x=GlobalMaxPooling1D()(x)
x=Dense(1,activation='sigmoid')(x)
model=Model(input_shape,x)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 100)       │    49,634,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 98, 32)         │         9,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 32, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 30, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 8, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,675,073 (189.50 MB)

 Trainable params: 49,675,073 (189.50 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history1=model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
    8/20000 ━━━━━━━━━━━━━━━━━━━━ 5:29 16ms/step - accuracy: 0.4980 - loss: 0.6939 

I0000 00:00:1715963208.309514     121 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20000/20000 ━━━━━━━━━━━━━━━━━━━━ 324s 16ms/step - accuracy: 0.6803 - loss: 0.5805 - val_accuracy: 0.6995 - val_loss: 0.5580
Epoch 2/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 318s 16ms/step - accuracy: 0.7089 - loss: 0.5459 - val_accuracy: 0.7013 - val_loss: 0.5569
Epoch 3/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 318s 16ms/step - accuracy: 0.7213 - loss: 0.5287 - val_accuracy: 0.6997 - val_loss: 0.5626
Epoch 4/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 318s 16ms/step - accuracy: 0.7336 - loss: 0.5101 - val_accuracy: 0.6984 - val_loss: 0.5668
Epoch 5/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 318s 16ms/step - accuracy: 0.7454 - loss: 0.4908 - val_accuracy: 0.6937 - val_loss: 0.5852
Epoch 6/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 318s 16ms/step - accuracy: 0.7570 - loss: 0.4710 - val_accuracy: 0.6894 - val_loss: 0.5928
Epoch 7/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 318s 16ms/step - accuracy: 0.7674 - loss: 0.4515 - val_accuracy: 0.6880 - val_loss: 0.6187
Epoch 8/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 318s 16ms/step - accur

In [18]:
loss, accuracy = model.evaluate(data_test, y_test)
print("Test Accuracy:", accuracy)

10000/10000 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.6762 - loss: 0.7147
Test Accuracy: 0.675378143787384


In [19]:
dim=100
input_shape=Input((T,))
x2=Embedding(V+1,dim)(input_shape)
x2=Conv1D(32,5,activation='relu')(x2)
x2=MaxPooling1D(2)(x2)
x2=Conv1D(64,5,activation='relu')(x2)
x2=MaxPooling1D(2)(x2)
x2=Conv1D(128,5,activation='relu')(x2)
x2=GlobalMaxPooling1D()(x2)
x2=Dense(1,activation='softmax')(x2)
model2=Model(input_shape,x2)
model2.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 100, 100)       │    49,634,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 96, 32)         │        16,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 48, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 44, 64)         │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 22, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 18, 128)        │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,701,953 (189.60 MB)

 Trainable params: 49,701,953 (189.60 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history2=model2.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (64, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


20000/20000 ━━━━━━━━━━━━━━━━━━━━ 330s 16ms/step - accuracy: 0.4993 - loss: 0.4906 - val_accuracy: 0.5016 - val_loss: 0.4567
Epoch 2/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 326s 16ms/step - accuracy: 0.4994 - loss: 0.4403 - val_accuracy: 0.5016 - val_loss: 0.4547
Epoch 3/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 326s 16ms/step - accuracy: 0.5000 - loss: 0.4177 - val_accuracy: 0.5016 - val_loss: 0.4596
Epoch 4/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 326s 16ms/step - accuracy: 0.4995 - loss: 0.3925 - val_accuracy: 0.5016 - val_loss: 0.4667
Epoch 5/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 326s 16ms/step - accuracy: 0.4999 - loss: 0.3678 - val_accuracy: 0.5016 - val_loss: 0.4818
Epoch 6/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 326s 16ms/step - accuracy: 0.4996 - loss: 0.3414 - val_accuracy: 0.5016 - val_loss: 0.5025
Epoch 7/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 326s 16ms/step - accuracy: 0.4984 - loss: 0.3172 - val_accuracy: 0.5016 - val_loss: 0.5353
Epoch 8/10
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 326s 16ms/step - accur

In [21]:
loss2, accuracy2 = model2.evaluate(data_test, y_test)
print("Test Accuracy:", accuracy2)

/opt/conda/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


10000/10000 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.5008 - loss: 0.6602
Test Accuracy: 0.5015812516212463


# prediction of random set of words from test set

In [46]:
def predict_random_words(model, tokenizer, test_data, test_labels, num_sentences=5, num_words=3):
    # Loop through a specified number of random sentences
    for _ in range(num_sentences):
        # Randomly select an index from the test set
        index = random.randint(0, len(test_data) - 1)
        
        # Select the text and corresponding label at the random index
        text = test_data.iloc[index]
        true_label = test_labels.iloc[index]
        
        # Preprocess the text
        preprocessed_text = preprocess(text)
        
        # Tokenize the preprocessed text
        tokenized_text = tokenizer.texts_to_sequences([preprocessed_text])[0]
        
        # If the tokenized text is empty, skip to avoid errors
        if not tokenized_text:
            print("Selected sentence is empty after preprocessing. Selecting a new sentence.")
            continue
        
        # Predict the sentiment for the entire sentence using the trained model
        sequence = tokenizer.texts_to_sequences([preprocessed_text])
        padded_sequence = pad_sequences(sequence, maxlen=T)
        prediction = model.predict(padded_sequence)[0][0]
        predicted_label = 1 if prediction >= 0.5 else 0
        
        # Print the true and predicted labels
        print("\nTrue Label (for the sentence):", true_label)
        print("Predicted Label (for the sentence):", predicted_label)
        print("Original Text:", text)
        print("Randomly Selected Words:")
        
        # Select multiple random words
        selected_words = random.sample(tokenized_text, min(num_words, len(tokenized_text)))
        for word_token in selected_words:
            # Retrieve the word from the tokenizer's index_word dictionary
            word = tokenizer.index_word.get(word_token, "Unknown word")
            print(word)
        
        print("-" * 50)

In [51]:
predict_random_words(model, tokenizer, df_test, y_test, num_sentences=5, num_words=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

True Label (for the sentence): 0
Predicted Label (for the sentence): 0
Original Text: angelvashir awww gots go
Randomly Selected Words:
go
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

True Label (for the sentence): 0
Predicted Label (for the sentence): 0
Original Text: ugh want dream accomplished already
Randomly Selected Words:
want
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

True Label (for the sentence): 1
Predicted Label (for the sentence): 1
Original Text: nberardi sarajchipps guess sara make mention mariage proposal site
Randomly Selected Words:
site
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

True Label (for the sentence): 1
Predicted Label (for the sentence): 1
Original Text: going beach tomorrow
Randomly Selected Words:
beach
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 